In [2]:
library(tidyverse)
library(readr)
library(igraph)
# get friends full series edgelist

edgefile_url <- "https://github.com/keithmcnulty/friends_analysis/blob/master/data/friends_full_series_edgelist.RDS?raw=true"
download.file(edgefile_url, "edgelist.RDS")

edgelist <- readRDS("edgelist.RDS")

knitr::kable(edgelist %>% head(10))



|from                |to           | weight|
|:-------------------|:------------|------:|
|a Casino Boss       |a Tourist    |      1|
|a Casino Boss       |Chandler     |      1|
|a Casino Boss       |Joey         |      1|
|a Casino Boss       |Phoebe       |      1|
|a Casino Boss       |Rachel       |      1|
|a Crew Member       |Alex         |      1|
|a Crew Member       |Chandler     |      1|
|a Crew Member       |Joey         |      1|
|a Crew Member       |The Director |      1|
|a Disembodied Voice |Phoebe       |      1|

In [5]:
friends <- c("Phoebe", "Monica", "Rachel", "Joey", "Ross", "Chandler")

In [15]:
edgelist_without <- edgelist %>% 
  dplyr::filter(!(from %in% friends & to %in% friends))

In [16]:
edgelist_matrix <- as.matrix(edgelist_without[ ,c("from", "to")])

In [18]:
friends_graph <- igraph::graph_from_edgelist(edgelist_matrix, directed = FALSE) %>% 
  igraph::set.edge.attribute("weight", value = edgelist_without$weight)

In [19]:
friends_graph

IGRAPH cde7282 UNW- 650 2961 -- 
+ attr: name (v/c), weight (e/n)
+ edges from cde7282 (vertex names):
 [1] a Casino Boss      --a Tourist          
 [2] a Casino Boss      --Chandler           
 [3] a Casino Boss      --Joey               
 [4] a Casino Boss      --Phoebe             
 [5] a Casino Boss      --Rachel             
 [6] a Crew Member      --Alex               
 [7] Chandler           --a Crew Member      
 [8] Joey               --a Crew Member      
+ ... omitted several edges

In [21]:
louvain_partition <- igraph::cluster_louvain(friends_graph)

In [25]:
friends_graph$community <- louvain_partition$membership

In [28]:
friends_graph

IGRAPH cde7282 UNW- 650 2961 -- 
+ attr: community (g/n), name (v/c), weight (e/n)
+ edges from cde7282 (vertex names):
 [1] a Casino Boss      --a Tourist          
 [2] a Casino Boss      --Chandler           
 [3] a Casino Boss      --Joey               
 [4] a Casino Boss      --Phoebe             
 [5] a Casino Boss      --Rachel             
 [6] a Crew Member      --Alex               
 [7] Chandler           --a Crew Member      
 [8] Joey               --a Crew Member      
+ ... omitted several edges

In [29]:
unique(friends_graph$community)

[1] 4 6 1 7 3 8 5 2

In [30]:
communities <- data.frame()

for (i in unique(friends_graph$community)) {
  # create subgraphs for each community
  subgraph <- induced_subgraph(friends_graph, v = which(friends_graph$community == i))
  # get size of each subgraph
  size <- igraph::gorder(subgraph)
  # get betweenness centrality
  btwn <-  igraph::betweenness(subgraph)
  communities <- communities %>% 
    dplyr::bind_rows(
      data.frame(community = i,
                 n_characters = size,
                 most_important = names(which(btwn == max(btwn)))
    )
  )
}

knitr::kable(communities %>% 
               dplyr::select(community, n_characters, most_important))



| community| n_characters|most_important  |
|---------:|------------:|:---------------|
|         4|          128|Chandler        |
|         6|          109|Joey            |
|         1|          105|Phoebe          |
|         7|           93|Rachel          |
|         3|          105|Monica          |
|         8|           94|Ross            |
|         5|            8|Intercom        |
|         2|            8|Bandleader      |
|         2|            8|Dennis Phillips |

In [32]:
top_five <- data.frame()

for (i in unique(friends_graph$community)) {
# create subgraphs for each community
  subgraph <- induced_subgraph(friends_graph, v = which(friends_graph$community == i))
  
  # for larger communities
  if (igraph::gorder(subgraph) > 20) {
    # get degree
    degree <-  igraph::degree(subgraph)
    # get top ten degrees
    top <- names(head(sort(degree, decreasing = TRUE), 5))
    result <- data.frame(community = i, rank = 1:5, character = top)
  } else {
    result <- data.frame(community = NULL, rank = NULL, character = NULL)
  }
  
  top_five <- top_five %>% 
    dplyr::bind_rows(result)
}

knitr::kable(
  top_five %>% 
    tidyr::pivot_wider(names_from = rank, values_from = character)
)



| community|1        |2            |3          |4        |5              |
|---------:|:--------|:------------|:----------|:--------|:--------------|
|         4|Chandler |Janice       |Bill       |Doug     |Security Guard |
|         6|Joey     |The Director |Gunther    |Director |Terry          |
|         1|Phoebe   |Guy          |Mike       |Steve    |Frank          |
|         7|Rachel   |Man          |Nurse      |Dr Long  |Amy            |
|         3|Monica   |Mr Geller    |Mrs Geller |Richard  |Pete           |
|         8|Ross     |Woman        |Carol      |Susan    |Charlie        |